### Student's t-test  

Assumptions:
- Normally distributed
- Homogeneous variance

Null hypothesis: The means of two samples are equal  

Unpaired

In [2]:
from scipy.stats import ttest_ind
import numpy as np

df1 = np.random.normal(20, 1, 23)
df2 = np.random.normal(20.8, 1, 23)

stat, p = ttest_ind(df1, df2)

print(f"T-Statistic: {stat:.2f}\np-value: {p:.3f}")

T-Statistic: -1.02
p-value: 0.314


Paired

In [3]:
from scipy.stats import ttest_rel

df1 = np.random.normal(20, 1, 23)
df2 = df1 + np.random.normal(0.6, 0.7, 23)

stat, p = ttest_rel(df1, df2)

print(f"T-Statistic: {stat:.2f}\np-value: {p:.3f}")

T-Statistic: -4.94
p-value: 0.000


One-sample

In [12]:
from scipy.stats import ttest_1samp
import warnings
warnings.filterwarnings("ignore")

stat, p = ttest_1samp(np.random.normal(3.1, 1, 8), 3)

print(f"T-Statistic: {stat:.2f}\np-value: {p:.3f}")

T-Statistic: -2.02
p-value: 0.083


### Analysis of Variance Test (ANOVA)  

Assumptions:
- Normally distributed
- Homogeneous variance

Null hypothesis: The means of samples are equal  

Unpaired

In [5]:
from scipy.stats import f_oneway

df1 = np.random.normal(20, 1, 23)
df2 = np.random.normal(20.2, 1, 23)
df3 = np.random.normal(19.8, 1, 23)
df4 = np.random.normal(20.3, 1, 23)

stat, p = f_oneway(df1, df2, df3, df4)

print(f"Statistic: {stat:.2f}\np-value: {p:.3f}")

Statistic: 0.93
p-value: 0.431


Paired

In [6]:
from statsmodels.stats.anova import AnovaRM
import pandas as pd

df1 = np.random.normal(20, 1, 23)
df2 = df1 + np.random.normal(0.6, 0.7, 23)
df3 = df1 + np.random.normal(0.1, 0.3, 23)

together = np.concatenate([df1, df2, df3])
d_from = [[i]*23 for i in range(3)]
d_from = d_from[0] + d_from[1] + d_from[2]
patient = list(range(23)) * 3

(df := pd.DataFrame({"drug":d_from, "patient":patient, "effect":together}))

,drug,patient,effect
0,0,0,20.274116
1,0,1,19.198682
2,0,2,20.689133
3,0,3,20.957441
4,0,4,21.999473
...,...,...,...
64,2,18,21.145065
65,2,19,19.682705
66,2,20,20.920504
67,2,21,21.420899


In [7]:
print(AnovaRM(df, "effect", "patient", within=["drug"]).fit())

              Anova
     F Value Num DF  Den DF Pr > F
----------------------------------
drug  7.0080 2.0000 44.0000 0.0023

